In [1]:
import boto3
print(f"Current boto3 version: {boto3.__version__}")

Current boto3 version: 1.35.90


In [2]:
import os
import time
import boto3
import json

# Initialize AWS clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=region)

# Generate a unique suffix using the current timestamp
# timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(time.time()))[-3:]
# suffix = f"{timestamp_str}"

account_id_suffix = account_id[:3]
region_suffix = ''.join([word[0] for word in region.split('-')])
suffix = f"{region_suffix}-{account_id_suffix}"

# Define Knowledge Base parameters
knowledge_base_name_standard = 'fixed-size-chunk-kb'
knowledge_base_name_hierarchical = 'hierarchical-chunk-kb'
knowledge_base_description = "Knowledge Base containing Amazon 10k financial documents"

print(f"AWS Region: {region}", f"Account ID: {account_id}", f"Suffix: {suffix}", sep='\n')

AWS Region: us-east-1
Account ID: 533267284022
Suffix: ue1-533


In [3]:
from utils.knowledgebase import BedrockKnowledgeBases
from utils.agents import BedrockAgents
from utils.agents import AWSResourceManager
from utils.chat import BedrockChat, SyntheticDataGenerator
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

    
kb = BedrockKnowledgeBases()
agents = BedrockAgents()

## Create Knowledge Base

##### To test the performance of Knowledge base's `Retrieve` and `RetrieveAndGenerate` APIs

In [4]:
kb_name = 'amazon-10k-kb'
vector_store = 'OPENSEARCH_SERVERLESS'
embedding_model = 'amazon.titan-embed-text-v2:0'

kb_id, ds_id, bucket_name = kb.create_knowledge_base(
    kb_name=kb_name, 
    kb_description='Knowledge Base containing Amazon 10K documents',
    vector_store=vector_store,
    embedding_model=embedding_model,
    suffix=suffix
)

2025-01-08 15:27:34,311 [INFO] ================================================================
2025-01-08 15:27:34,312 [INFO] Step 1: Create S3 bucket
2025-01-08 15:27:34,313 [INFO] Creating S3 bucket: amazon-10k-kb-ue1-533-bucket
2025-01-08 15:27:34,707 [INFO] Created S3 bucket: amazon-10k-kb-ue1-533-bucket
2025-01-08 15:27:34,708 [INFO] ================================================================
2025-01-08 15:27:34,708 [INFO] Step 2: Create KB execution role
2025-01-08 15:27:34,709 [INFO] Creating Bedrock KB execution role: amazon-10k-kb-ue1-533-execution-role
2025-01-08 15:27:35,584 [INFO] ================================================================
2025-01-08 15:27:35,586 [INFO] Step 3: Create AOSS encryption/network/access policies
2025-01-08 15:27:35,587 [INFO] Creating encryption, network, and access policies for azon-10k-kb-ue1-533-vector-store
2025-01-08 15:27:36,283 [INFO] Created AOSS policies: b-ue1-533-vector-store-en-policy, b-ue1-533-vector-store-ne-policy, b-u

In [5]:
print(f"Knowledge base ID: {kb_id}, Data source ID: {ds_id}, Bucket name: {bucket_name}")

Knowledge base ID: RZJXH5WHWU, Data source ID: KS6SL2YJ2K, Bucket name: amazon-10k-kb-ue1-533-bucket


In [6]:
# Delete the Knowledge Base and all assoicated resources based on kb name and suffix
# kb.delete_knowledge_base_resources_by_name(kb_name=kb_name, suffix=suffix)

### Upload KB documents to S3  

In [7]:
def upload_documents_to_s3(bucket_name, folder_path):
    s3_client = boto3.client('s3', region_name=region)
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file not in ["LICENSE", "NOTICE", "README.md"]:
                print(f"Uploading {file_path} to {bucket_name}")
                s3_client.upload_file(file_path, bucket_name, file)
            else:
                print(f"Skipping {file_path}")

# Example usage
upload_documents_to_s3(bucket_name, "kb_documents")

Uploading kb_documents/AMAZON_2022_10K.pdf to amazon-10k-kb-ue1-533-bucket


### Initiate Knowledge Base Data Ingestion 

In [8]:
kb.synchronize_data(kb_id=kb_id, ds_id=ds_id)

2025-01-08 15:33:43,629 [INFO] Starting data synchronization for KB ID: RZJXH5WHWU and DS ID: KS6SL2YJ2K
2025-01-08 15:33:44,760 [INFO] Started ingestion job: {'dataSourceId': 'KS6SL2YJ2K', 'ingestionJobId': 'JN2Q9VXWQ8', 'knowledgeBaseId': 'RZJXH5WHWU', 'startedAt': datetime.datetime(2025, 1, 8, 23, 33, 44, 756897, tzinfo=tzutc()), 'statistics': {'numberOfDocumentsDeleted': 0, 'numberOfDocumentsFailed': 0, 'numberOfDocumentsScanned': 0, 'numberOfMetadataDocumentsModified': 0, 'numberOfMetadataDocumentsScanned': 0, 'numberOfModifiedDocumentsIndexed': 0, 'numberOfNewDocumentsIndexed': 0}, 'status': 'STARTING', 'updatedAt': datetime.datetime(2025, 1, 8, 23, 33, 44, 756897, tzinfo=tzutc())}
2025-01-08 15:33:44,891 [INFO] Ingestion job status: STARTING
.....
2025-01-08 15:33:50,097 [INFO] Ingestion job status: IN_PROGRESS
.....
2025-01-08 15:33:55,266 [INFO] Ingestion job status: IN_PROGRESS
.....
2025-01-08 15:34:00,435 [INFO] Ingestion job status: COMPLETE
.....
2025-01-08 15:34:05,464 [

## Create Agent Integrated with Knowledge Base

##### To test the performance of an agent when integrated with the knowledge base.

In [9]:
agent_name=f'rag-agent-{suffix}'
print('Agent name:', agent_name)

Agent name: rag-agent-ue1-533


In [10]:
agent_description = """You are a versatile AI assistant that combines provided context with general knowledge to deliver accurate, 
well-sourced information and solutions to questions across various domains."""


agent_instruction = """You are a General AI Assistant helping users with diverse tasks and questions across multiple domains. Your role is to 
leverage both your general expertise and context-based knowledge retrieved in real-time to provide accurate, relevant, and up-to-date information. You are capable of 
answering questions with clear explanations on a broad range of topics, including but not limited to technology, science, health, finance, and general knowledge.

Response style:
1. Maintain a professional yet friendly tone
2. Use clear, straightforward language
3. Focus on practical, actionable solutions
4. Structure responses logically and concisely
5. Break down complex concepts simply
6. Stay relevant to user queries
7. Avoid unnecessary information
8. Ensure a natural conversation flow
9. Use available information before asking questions
10. Do not directly quote or mention that the information is from the context."""


In [13]:
rag_agent = agents.create_bedrock_agent(
    agent_name=agent_name, 
    agent_description=agent_description,
    agent_instructions=agent_instruction,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0", # TODO: this should probably be a list
    kb_id=kb_id,
    associate_kb=True,
    code_interpreter=True,  # Enabling code interpreter to make the agent use instructions
    kb_usage_description="Use this knowledge base to provide responses to user queries on Amazon's financial performance and business activities",
    verbose=True
    )


2025-01-08 15:39:35,755 [INFO] Creating IAM role: rag-agent-ue1-533-exec-role
2025-01-08 15:39:35,880 [INFO] Created IAM role: arn:aws:iam::533267284022:role/rag-agent-ue1-533-exec-role
2025-01-08 15:39:41,104 [INFO] Created managed policy: arn:aws:iam::533267284022:policy/rag-agent-ue1-533-AmazonBedrockAgentModelAccessPolicy
2025-01-08 15:39:41,232 [INFO] Attached policy 'rag-agent-ue1-533-AmazonBedrockAgentModelAccessPolicy' to role 'rag-agent-ue1-533-exec-role'
2025-01-08 15:39:41,435 [INFO] Created agent: KPJOGYOMCS
2025-01-08 15:39:56,441 [INFO] Updating policy 'rag-agent-ue1-533-AmazonBedrockAgentKBAccessPolicy' for role 'arn:aws:iam::533267284022:role/rag-agent-ue1-533-exec-role'
2025-01-08 15:39:56,658 [INFO] Created new policy: rag-agent-ue1-533-AmazonBedrockAgentKBAccessPolicy
2025-01-08 15:39:56,793 [INFO] Attached policy 'rag-agent-ue1-533-AmazonBedrockAgentKBAccessPolicy' to role 'rag-agent-ue1-533-exec-role'
2025-01-08 15:39:56,794 [INFO] Associating agent 'KPJOGYOMCS' wi

In [14]:
agents.prepare_bedrock_agent(agent_name=agent_name, verbose=True)

..........
2025-01-08 15:44:13,075 [INFO] Getting status for agent 'rag-agent-ue1-533'
2025-01-08 15:44:13,341 [INFO] Agent 'rag-agent-ue1-533' status: {'agentId': 'KPJOGYOMCS', 'agentName': 'rag-agent-ue1-533', 'agentStatus': 'PREPARED', 'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0', 'agentCollaboration': 'DISABLED', 'orchestrationType': 'DEFAULT', 'createdAt': datetime.datetime(2025, 1, 8, 23, 39, 41, 410378, tzinfo=tzutc()), 'lastUpdatedAt': datetime.datetime(2025, 1, 8, 23, 39, 41, 841053, tzinfo=tzutc())}
2025-01-08 15:44:13,342 [INFO] Agent rag-agent-ue1-533 is now prepared


In [12]:
# agents.delete_bedrock_agent(agent_name=agent_name, delete_role=True, verbose=True)

2025-01-08 15:39:19,365 [INFO] Found agent 'rag-agent-ue1-533' with ID: LPBVM9TI8Z
2025-01-08 15:39:19,366 [INFO] Deleting aliases for agent LPBVM9TI8Z...
2025-01-08 15:39:19,491 [INFO] Deleting alias TSTALIASID from agent LPBVM9TI8Z
2025-01-08 15:39:19,643 [INFO] Deleting action groups for agent LPBVM9TI8Z...
2025-01-08 15:39:19,776 [INFO] Deleting action group 'CodeInterpreterAction' (ID: LUFRZUP8VW) from agent LPBVM9TI8Z
2025-01-08 15:39:19,913 [INFO] Action group details: {'ResponseMetadata': {'RequestId': '084420e0-0446-4de4-bc36-795af417ae6a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 08 Jan 2025 23:39:19 GMT', 'content-type': 'application/json', 'content-length': '360', 'connection': 'keep-alive', 'x-amzn-requestid': '084420e0-0446-4de4-bc36-795af417ae6a', 'x-amz-apigw-id': 'EF7qSEHooAMEIlw=', 'x-amzn-trace-id': 'Root=1-677f0ca7-1d8da784630b2bd71380ec16'}, 'RetryAttempts': 0}, 'agentActionGroup': {'actionGroupId': 'LUFRZUP8VW', 'actionGroupName': 'CodeInterpreterActio

In [15]:
rag_agent

('KPJOGYOMCS',
 'TSTALIASID',
 'arn:aws:bedrock:us-east-1:533267284022:agent-alias/KPJOGYOMCS/TSTALIASID')

In [17]:
print(f"Agent name: {agent_name}", f"Agent ID: {rag_agent[0]}", f"Agent Alias ID: {rag_agent[1]}", sep='\n')

Agent name: rag-agent-ue1-533
Agent ID: KPJOGYOMCS
Agent Alias ID: TSTALIASID


## Generate response using KB's `RetrieveAndGenerate` API

In [9]:
user_query = "What is the total cost of Octank Financial's property and equipment as of December 31, 2022?"

In [10]:
def retrieve_from_knowledge_base(knowledge_base_id, query, number_of_results=3):
    response = bedrock_agent_runtime_client.retrieve(
        knowledgeBaseId=knowledge_base_id,
        retrievalQuery={"text": query},
        retrievalConfiguration={
            "vectorSearchConfiguration": {
                "numberOfResults": number_of_results
            }
        }
    )
    return response['retrievalResults']

In [11]:
system_prompt = """Human: You are a versatile AI assistant. Your task is to combine the provided context with your general knowledge to deliver accurate, well-sourced information and solutions to questions across various domains. Use the context and your general expertise to answer user queries effectively.

Here is the context and user's question:
<context>
$search_results$
</context>

<question>
$query$
</question>

Your role is to leverage both your general expertise and context-based knowledge to provide accurate, relevant, and up-to-date information. Follow these guidelines in your responses:
1. Maintain a professional yet friendly tone.
2. Use clear, straightforward language.
3. Focus on practical, actionable solutions.
4. Structure responses logically and concisely.
5. Break down complex concepts simply.
6. Stay relevant to user queries.
7. Avoid unnecessary information.
8. Ensure a natural conversation flow.
9. Use available information before asking questions.
10. Do not directly quote or mention that the information is from the context.

Assistant:
"""


In [12]:
def retrieve_and_generate_from_kb(query, kb_id, num_results=5, prompt_template=None, model_id="anthropic.claude-v2", region="us-east-1"):
    """
    Retrieves information from a knowledge base and generates a response using Amazon Bedrock.
    
    Args:
        query (str): The input query text
        kb_id (str): The knowledge base ID
        num_results (int, optional): Number of results to retrieve. Defaults to 5.
        prompt_template (str, optional): Custom prompt template for generation. Defaults to None.
        model_id (str, optional): Foundation model ID. Defaults to "anthropic.claude-v2".
        region (str, optional): AWS region. Defaults to "us-east-1".
    
    Returns:
        str: The generated response text
        
    Raises:
        Exception: If there's an error during the retrieve and generate operation
    """
    try:
        # Construct the base configuration
        config = {
            'knowledgeBaseId': kb_id,
            'modelArn': f'arn:aws:bedrock:{region}::foundation-model/{model_id}',
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': num_results,
                    'overrideSearchType': 'SEMANTIC'
                }
            }
        }
        
        # Add prompt template if provided
        if prompt_template:
            config['generationConfiguration'] = {
                'promptTemplate': {
                    'textPromptTemplate': prompt_template
                }
            }
            
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={
                'text': query
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': config
            }
        )
        
        return response['output']['text']
    
    except Exception as e:
        print(f"Error during retrieve and generate operation: {str(e)}")
        raise

In [13]:
response_text = retrieve_and_generate_from_kb(
    kb_id=kb_id,
    query=user_query,
    region="us-east-1",
    num_results=5,
    prompt_template=system_prompt,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0"
)

print_markdown(response_text)


According to the table provided in the notes to Octank Financial's consolidated financial statements, the total cost of the company's property and equipment as of December 31, 2022 is $440,000,000. This amount represents the sum of the costs across various asset classes, including land ($50,000,000), buildings ($200,000,000), machinery and equipment ($150,000,000), furniture and fixtures ($20,000,000), and vehicles ($15,000,000).

It's important to note that the total cost does not reflect the accumulated depreciation or the net book value of these assets. The net book value, which considers the impact of depreciation, is reported as $307,500,000 in the table. The cost figure represents the original acquisition cost or construction cost of these long-lived assets before any depreciation has been applied.

In [14]:
user_query = "How is the net book value of each asset class calculated?"

response_text = retrieve_and_generate_from_kb(
    kb_id=kb_id,
    query=user_query,
    region="us-east-1",
    num_results=5,
    prompt_template=system_prompt,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0"
)

print_markdown(response_text)


The net book value of each asset class is calculated by taking the original cost of the asset and subtracting the accumulated depreciation from it.

To break it down:

Cost - This is the original purchase price or cost to acquire the asset.

Accumulated Depreciation - This is the total amount of depreciation expense that has been recorded for the asset since it was put into service. Depreciation is the systematic allocation of the cost of an asset over its estimated useful life.

Net Book Value = Cost - Accumulated Depreciation

For example, let's look at the Buildings asset class from the table:

Cost: $200,000,000
Accumulated Depreciation: $40,000,000

Net Book Value = $200,000,000 - $40,000,000 = $160,000,000

So the net book value of the Buildings is $160,000,000. This represents the remaining cost of the buildings that has not yet been depreciated.

The net book value calculation is done for each asset class to show the remaining undepreciated cost that can provide future economic benefits to the company over the assets' useful lives.

## Create RAG Agent

In [6]:
agent_name=f'rag-agent-{suffix}'
print('Agent name:', agent_name)

kb_id = "892DWWRLXW"

Agent name: rag-agent-ue1-533


In [8]:
rag_agent = agents.create_bedrock_agent(
    agent_name=agent_name, 
    agent_description=agent_description,
    agent_instructions=agent_instruction,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    kb_id=kb_id,
    associate_kb=True,
    code_interpreter=True,
    kb_usage_description="Use this knowledge base to provide responses to user queries on any topic",
    verbose=True
    )

2025-01-07 23:34:21,077 [INFO] Creating IAM role: rag-agent-ue1-533-exec-role
2025-01-07 23:34:21,205 [INFO] Created IAM role: arn:aws:iam::533267284022:role/rag-agent-ue1-533-exec-role
2025-01-07 23:34:26,441 [INFO] Created managed policy: arn:aws:iam::533267284022:policy/rag-agent-ue1-533-AmazonBedrockAgentModelAccessPolicy
2025-01-07 23:34:26,578 [INFO] Attached policy 'rag-agent-ue1-533-AmazonBedrockAgentModelAccessPolicy' to role 'rag-agent-ue1-533-exec-role'
2025-01-07 23:34:26,780 [INFO] Created agent: LPBVM9TI8Z
2025-01-07 23:34:41,785 [INFO] Updating policy 'rag-agent-ue1-533-AmazonBedrockAgentKBAccessPolicy' for role 'arn:aws:iam::533267284022:role/rag-agent-ue1-533-exec-role'
2025-01-07 23:34:42,003 [INFO] Created new policy: rag-agent-ue1-533-AmazonBedrockAgentKBAccessPolicy
2025-01-07 23:34:42,137 [INFO] Attached policy 'rag-agent-ue1-533-AmazonBedrockAgentKBAccessPolicy' to role 'rag-agent-ue1-533-exec-role'
2025-01-07 23:34:42,139 [INFO] Associating agent 'LPBVM9TI8Z' wi

In [7]:
# agents.delete_bedrock_agent(agent_name=agent_name, delete_role=True, verbose=True)

2025-01-07 23:34:02,967 [INFO] Found agent 'rag-agent-ue1-533' with ID: R83CGJO0JB
2025-01-07 23:34:02,967 [INFO] Deleting aliases for agent R83CGJO0JB...
2025-01-07 23:34:03,090 [INFO] Deleting alias TSTALIASID from agent R83CGJO0JB
2025-01-07 23:34:03,264 [INFO] Deleting action groups for agent R83CGJO0JB...
2025-01-07 23:34:03,393 [INFO] Deleting action group 'CodeInterpreterAction' (ID: OFZ6PEQWUU) from agent R83CGJO0JB
2025-01-07 23:34:03,523 [INFO] Action group details: {'ResponseMetadata': {'RequestId': '0165e82f-dd39-445c-bb99-886afd5a38ba', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 08 Jan 2025 07:34:03 GMT', 'content-type': 'application/json', 'content-length': '360', 'connection': 'keep-alive', 'x-amzn-requestid': '0165e82f-dd39-445c-bb99-886afd5a38ba', 'x-amz-apigw-id': 'EDuQ2EC0IAMEn7Q=', 'x-amzn-trace-id': 'Root=1-677e2a6b-0af4d66271f4a3b479d9d091'}, 'RetryAttempts': 0}, 'agentActionGroup': {'actionGroupId': 'OFZ6PEQWUU', 'actionGroupName': 'CodeInterpreterActio

In [12]:
agents.prepare_bedrock_agent(agent_name=agent_name, verbose=True)

2025-01-07 18:46:29,359 [INFO] Agent 'rag-agent-ue1-533' status: {'agentId': '52NEUIMYVP', 'agentName': 'rag-agent-ue1-533', 'agentStatus': 'PREPARING', 'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0', 'agentCollaboration': 'DISABLED', 'orchestrationType': 'DEFAULT', 'createdAt': datetime.datetime(2025, 1, 8, 2, 46, 3, 631679, tzinfo=tzutc()), 'lastUpdatedAt': datetime.datetime(2025, 1, 8, 2, 46, 4, 40477, tzinfo=tzutc())}
2025-01-07 18:46:29,360 [INFO] Agent preparing... (attempt 1/12)
.....
2025-01-07 18:46:34,611 [INFO] Agent 'rag-agent-ue1-533' status: {'agentId': '52NEUIMYVP', 'agentName': 'rag-agent-ue1-533', 'agentStatus': 'PREPARED', 'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0', 'agentCollaboration': 'DISABLED', 'orchestrationType': 'DEFAULT', 'createdAt': datetime.datetime(2025, 1, 8, 2, 46, 3, 631679, tzinfo=tzutc()), 'lastUpdatedAt': datetime.datetime(2025, 1, 8, 2, 46, 4, 40477, tzinfo=tzutc())}
2025-01-07 18:46:34,612 [INFO] Agent rag-agent-ue1

In [13]:
user_query = "What is the total cost of Octank Financial's property and equipment as of December 31, 2022?"

result = agents.invoke(
    agent_name=agent_name, 
    input_text=user_query, 
    verbose=True, 
    trace_level='core',
    save_trace_json_file='agent_full_trace.json'
)

2025-01-07 18:46:46,237 [INFO] Invoking agent 'rag-agent-ue1-533' with input: What is the total cost of Octank Financial's property and equipment as of December 31, 2022?
---- Step 1 ----
Took 2.6s, using 3251 tokens (in: 3145, out: 106) to complete action.
Final response:
According to the table for Property and Equipment, Net under NOTES TO CONSOLIDATED FINANCIAL STATEMENTS, the total cost of Octank Financial's property and equipment as of December 31, 2022 is $440,000,000....
Agent made a total of 1 LLM calls, using 3251 tokens (in: 3145, out: 106), and took 3.0 total seconds.


In [13]:
user_query = "What is the total cost of Octank Financial's property and equipment as of December 31, 2022?"

result = agents.new_invoke(
    agent_name=agent_name, 
    input_text=user_query, 
    verbose=True, 
    trace_level='core',
    # save_trace_json_file='agent_full_trace.json'
)

2025-01-07 23:37:14,244 [INFO] Invoking agent 'rag-agent-ue1-533' with input: What is the total cost of Octank Financial's property and equipment as of December 31, 2022?
---- Step 1 ----
Took 3.1s, using 1825 tokens (in: 1679, out: 146) to complete action.
To find the total cost of Octank Financial's property and equipment as of December 31, 2022, I will need to search for information on Octank Financial's financial statements or reports. Let me query the knowledge base for relevant information.
---- Step 2 ----
Took 4.8s, using 2122 tokens (in: 2019, out: 103) to complete action.
Final response:


According to the table for Property and Equipment, Net under NOTES TO CONSOLIDATED FINANCIAL STATEMENTS, the total cost of Octank Financial's property and equipment as of December 31, 2022 was $440,000,000.


...
Agent made a total of 2 LLM calls, using 3947 tokens (in: 3698, out: 249), and took 8.1 total seconds.


In [10]:
print_markdown(result['response'])



According to the table for Property and Equipment, Net under NOTES TO CONSOLIDATED FINANCIAL STATEMENTS, the total cost of Octank Financial's property and equipment as of December 31, 2022 was $440,000,000.




In [15]:
result['raw_trace'][0]

{'agentAliasId': 'TSTALIASID',
 'agentId': 'LPBVM9TI8Z',
 'agentVersion': 'DRAFT',
 'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:533267284022:agent-alias/LPBVM9TI8Z/TSTALIASID'}],
 'sessionId': '3d08dcfd-d1ed-4885-a911-b9991282eb1f',
 'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048,
     'stopSequences': ['</function_calls>', '</answer>', '</error>'],
     'temperature': 0.0,
     'topK': 250,
     'topP': 1.0},
    'text': '{"system":"You are a General AI Assistant helping users with diverse tasks and questions across multiple domains. Your role is to leverage both your general expertise and context-based knowledge retrieved in real-time to provide accurate, relevant, and up-to-date information. You are capable of answering questions with clear explanations on a broad range of topics, including but not limited to technology, science, health, finance, and general knowledge.Response style:1. Maintain a profession

In [17]:
# Write to JSON file with proper formatting
with open('output.json', 'w') as f:
    json.dump(result['raw_trace'], f, indent=2)

# Print formatted JSON
print(json.dumps(result['raw_trace'], indent=2))

[
  {
    "agentAliasId": "TSTALIASID",
    "agentId": "LPBVM9TI8Z",
    "agentVersion": "DRAFT",
    "callerChain": [
      {
        "agentAliasArn": "arn:aws:bedrock:us-east-1:533267284022:agent-alias/LPBVM9TI8Z/TSTALIASID"
      }
    ],
    "sessionId": "3d08dcfd-d1ed-4885-a911-b9991282eb1f",
    "trace": {
      "orchestrationTrace": {
        "modelInvocationInput": {
          "inferenceConfiguration": {
            "maximumLength": 2048,
            "stopSequences": [
              "</function_calls>",
              "</answer>",
              "</error>"
            ],
            "temperature": 0.0,
            "topK": 250,
            "topP": 1.0
          },
          "text": "{\"system\":\"You are a General AI Assistant helping users with diverse tasks and questions across multiple domains. Your role is to leverage both your general expertise and context-based knowledge retrieved in real-time to provide accurate, relevant, and up-to-date information. You are capable of answ